In [1]:
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path, get_plate_number, get_postion_number,get_begin_index
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae, plot_raw_plus, get_hyphae
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from amftrack.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children, resolve_ambiguity, relabel_nodes_after_amb
from collections import Counter
from IPython.display import clear_output
from amftrack.notebooks.analysis.data_info import *
from amftrack.pipeline.functions.node_id import reconnect_degree_2
from amftrack.pipeline.functions.extract_graph import (
    generate_skeleton,
    from_nx_to_tab,
    prune_graph
)

from amftrack.pipeline.functions.hyphae_id_surf import (
    clean_and_relabel,
    get_mother,
    save_hyphaes,
    resolve_ambiguity_two_ends,
    clean_obvious_fake_tips,
    width_based_cleaning
)
from amftrack.pipeline.functions.experiment_class_surf import clean_exp_with_hyphaes
from scipy.ndimage import gaussian_filter
from scipy import ndimage
from scipy.optimize import curve_fit
from shapely.geometry import Polygon, shape
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path, get_plate_number, get_postion_number,get_begin_index
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae, plot_raw_plus, get_hyphae
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from amftrack.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children, resolve_ambiguity, relabel_nodes_after_amb
from collections import Counter
from IPython.display import clear_output
from amftrack.notebooks.analysis.data_info import *
from amftrack.pipeline.functions.node_id import reconnect_degree_2
from amftrack.pipeline.functions.extract_graph import (
    generate_skeleton,
    from_nx_to_tab,
    prune_graph
)

from amftrack.pipeline.functions.hyphae_id_surf import (
    clean_and_relabel,
    get_mother,
    save_hyphaes,
    resolve_ambiguity_two_ends,
    clean_obvious_fake_tips,
    width_based_cleaning
)
from amftrack.pipeline.functions.experiment_class_surf import clean_exp_with_hyphaes
from scipy.ndimage import gaussian_filter
from scipy import ndimage
from scipy.optimize import curve_fit
from shapely.geometry import Polygon, shape
directory = directory_project
plate_number =94
plate = get_postion_number(plate_number)
begin =269
end = 339
dates_datetime = get_dates_datetime(directory,plate)
dates = dates_datetime[begin:end+1]
exp = Experiment(plate,directory)
exp.load(dates) #for method 2
lengths = [get_length(exp,t) for t in range(exp.ts)]
times = [get_time(exp,0,t) for t in range(exp.ts)]
lengths = np.array(lengths)
times = np.array(times)
areas = []   
for t in range(exp.ts):
    nodes = np.array([node.pos(t) for node in exp.nodes if node.is_in(t)])
    hull=scipy.spatial.ConvexHull(nodes)
    poly = Polygon([nodes[vertice] for vertice in hull.vertices])
    areas.append(poly.area* 1.725**2/(1000**2))
areas = np.array(areas)
speeds = (lengths[1:]-lengths[:-1])/(times[1:]-times[:-1])
speeds_area = (areas[1:]-areas[:-1])/(times[1:]-times[:-1])

end = 30
begin = 0
lengths_select = lengths[begin:end]
times_select = times[begin:end]
areas_select = areas[begin:end]

coefs = np.polyfit(times_select,areas_select ,2)
p = np.poly1d(coefs)
def func(t,a,t0):
    return(a*(t-t0)**2)
popt0, pcov = curve_fit(func, times_select,areas_select ,bounds = ([0,-np.inf],2*[np.inf]),p0=[1,1])
def func2(t,alpha,C,d):
    return(C*(np.exp(alpha*(t)))+d)
popt1, pcov = curve_fit(func2, times_select, lengths_select/1000,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])


In [2]:
def func3(t):
    return(func2(t,*popt1)/p(t))
plt.rcParams.update({
    "font.family": "verdana",
'font.weight' : 'normal',
'font.size': 20})
fig=plt.figure(figsize=(9,8))
ax = fig.add_subplot(111)
# ax.plot(times,func(np.array(times),*popt0),label = f'quadratic fit a={"{:.2f}".format(popt0[0])} mm^2.h-2')
ax.set_xlabel('time (h)')
ax.set_ylabel('network density $(mm.mm^{-2})$')
ax.scatter(times,lengths/1000/areas)
ax.plot(times,func3(np.array(times)),label = f'exponential fit : alpha= {"{:.2f}".format(popt1[0]*24)}.day-1',color='red')


# plt.legend(loc = 'upper right')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
data = pd.DataFrame(list(zip(speeds,speeds_area,times[:-1])),columns = ['speed','speed_area','time'])
# data['straight'] = data['straight_distance']*1.725/data['curv_growth']
# data['straight_scaled'] = data['straight_distance']*1.725/np.sqrt(data['curv_growth'])
# data['time'] = lapses_time
# data['straight_time'] = data['straight']/data['time']
# data['speed'] = data['curv_growth']/data['time']
correct = data.loc[data['speed']<50000]

In [4]:
plt.close('all')
abcisse = 'time'
ordinate = 'speed'
tab = correct
baits_sort = tab.sort_values(abcisse)
N=8
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time (h)')

ax.set_ylabel('total network length growth growth speed ($\mu m .h^{-1}$)',color='red')
# ax.set_xlim(-0.5,0.5)
ax.set_ylim(0,50000)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))

ax.scatter(tab[abcisse],tab[ordinate],alpha=0.3,color='red')
ax.plot(moving_av[abcisse],moving_av[ordinate],color='red',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std',linestyle = 'dotted')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std',linestyle = 'dotted')
ax.plot(moving_av[abcisse],1000*popt1[0]*popt1[1]*np.exp(popt1[0]*moving_av[abcisse]),color='red',linestyle='dashdot')
ax.spines['right'].set_color('red')
ax.tick_params(axis='y', colors='red')  #setting up Y-axis tick color to black
ax2 = ax.twinx()
abcisse = 'time'
ordinate = 'speed_area'
tab = correct
baits_sort = tab.sort_values(abcisse)
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()
ax2.spines['right'].set_color('blue')
ax2.tick_params(axis='y', colors='blue')  #setting up Y-axis tick color to black
ax2.set_ylabel('Network area growth speed ($\mu m^2 .h^{-1}$)',color='blue')
# ax.set_xlim(-0.5,0.5)
# ax.set_ylim(-0.5,0.5)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))

ax2.scatter(tab[abcisse],tab[ordinate],alpha=0.3,color='blue')
ax2.plot(moving_av[abcisse],moving_av[ordinate],color='blue',label = 'moving average')
ax2.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='blue',label = 'std',linestyle = 'dotted')
ax2.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='blue',label = 'std',linestyle = 'dotted')
ax2.plot(moving_av[abcisse],2*coefs[0]*moving_av[abcisse]+coefs[1],color='blue',linestyle = 'dashdot')


/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:1377: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x[:, None]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:237: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x = x[:, np.newaxis]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:239: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]


In [16]:
coefs[1]

4.2855994221866816

In [31]:
plt.close('all')
abcisse = 'time'
ordinate = 'speed_area'
tab = correct
baits_sort = tab.sort_values(abcisse)
N=4
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time (h)')

ax.set_ylabel('Network scale growth speed ($\mu m .h^{-1}$)')
# ax.set_xlim(-0.5,0.5)
# ax.set_ylim(-0.5,0.5)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))

ax.scatter(tab[abcisse],tab[ordinate],alpha=0.3)
ax.plot(moving_av[abcisse],moving_av[ordinate],color='green',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …